In [1]:
import tensorflow as tf
from tensorflow import keras 

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

In [3]:
housing = fetch_california_housing()

In [4]:
X_train_full, X_test, y_train_full, y_test = train_test_split( housing.data,
         housing.target.reshape(-1, 1), random_state=42)

X_train, X_valid, y_train, y_valid =  train_test_split( X_train_full, 
        y_train_full, random_state=42)

In [5]:
scaler = StandardScaler()

In [6]:
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# Custom Loss Function - Huber Loss

In [7]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [8]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation= "selu", 
       kernel_initializer= "lecun_normal", 
       input_shape=X_train.shape[1:]),
    keras.layers.Dense(1),
])

In [14]:
model.compile(loss=huber_fn, optimizer="nadam", metrics=["mae"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                270       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


# Train the Model

In [15]:
print(X_train_scaled.shape, y_train.shape, X_valid_scaled.shape, y_valid.shape)

(11610, 8) (11610, 1) (3870, 8) (3870, 1)


In [16]:
model.fit(X_train_scaled, y_train, epochs=15,
    validation_data=(X_valid_scaled, y_valid))

Epoch 1/15
363/363 [==============================] - 1s 2ms/step - loss: 0.5696 - mae: 0.9316 - val_loss: 0.2076 - val_mae: 0.4994
Epoch 2/15
363/363 [==============================] - 1s 2ms/step - loss: 0.2081 - mae: 0.5026 - val_loss: 0.1944 - val_mae: 0.4767
Epoch 3/15
363/363 [==============================] - 1s 2ms/step - loss: 0.1995 - mae: 0.4886 - val_loss: 0.2197 - val_mae: 0.5022
Epoch 4/15
363/363 [==============================] - 1s 2ms/step - loss: 0.1962 - mae: 0.4830 - val_loss: 0.1990 - val_mae: 0.4844
Epoch 5/15
363/363 [==============================] - 1s 2ms/step - loss: 0.1922 - mae: 0.4776 - val_loss: 0.1930 - val_mae: 0.4724
Epoch 6/15
363/363 [==============================] - 1s 2ms/step - loss: 0.1895 - mae: 0.4737 - val_loss: 0.1822 - val_mae: 0.4591
Epoch 7/15
363/363 [==============================] - 1s 2ms/step - loss: 0.1867 - mae: 0.4690 - val_loss: 0.1739 - val_mae: 0.4523
Epoch 8/15
363/363 [==============================] - 1s 2ms/step - loss: 0.

# Evaluate Model

In [19]:
model.evaluate(X_test_scaled, y_test)

162/162 [==============================] - 0s 1ms/step - loss: 0.1762 - mae: 0.4521


[0.17624062299728394, 0.45205244421958923]